In [1]:
import pyomo.environ as pyo
import pandas as pd
from pathlib import Path
import preprocess_data as ppd
from itertools import zip_longest

# from Camm18 paper to verify obtained results later
known_x_sols = [[8, 14, 17, 19, 29, 33, 35, 38, 51, 55, 59, 68, 75, 78, 87],
              [4, 8, 14, 18, 29, 33, 35, 38, 51, 55, 59, 68, 75, 78, 87],
              [8, 14, 17, 18, 29, 33, 35, 38, 51, 55, 59, 68, 75, 78, 87]] 

## Data

In [2]:
data_path = Path("../data/")
output_path = Path("../output/")
df, adjacent_matrix = ppd.get_df_adj(data_path, 2021)
model = pyo.AbstractModel()

## Defs

In [3]:
def param_adjacent(m, i, j):
    return int(j in adjacent_matrix[i])

def param_pop_2010(m, i):
    return df["population_2010"][i - 1]

def param_pop_2020(m, i):
    return df["population_2010"][i - 1]

def con_a(m, i):
    return sum((m.a[i, j] * m.x[j]) for j in m.J) >= m.y[i]

def con_x(m):
    return sum(m.x[j] for j in m.J) <= m.k

def obj_sum(m):
    return pyo.summation(m.p, m.y)

## Constraints

In [4]:
# value of n (number of counties)
model.n = 88

# TOSET limit on number of pirincipal places of buisnesses opened (init to 5)
model.k = 15

# range of i and j (iterating over counties)
model.I = pyo.RangeSet(1, model.n)
model.J = pyo.RangeSet(1, model.n)

model.p = pyo.Param(model.I, initialize=param_pop)  # population of county i

# model.a = pyo.Set(model.I, model.J, within=pyo.Binary, initialize=param_adjacent)  # 1 if county i and j are adjacent
model.a = pyo.Param(model.I, model.J, domain=pyo.Binary, initialize=param_adjacent)  # 1 if county i and j are adjacent

model.x = pyo.Var(model.J, domain=pyo.Binary)  # 1 if principal place of business is opened in county j
model.y = pyo.Var(model.I, domain=pyo.Binary)  # 1 if county i is covered

model.obj = pyo.Objective(rule=obj_sum, sense=pyo.maximize)

model.a_constraint = pyo.Constraint(model.I, rule=con_a)
model.x_constraint = pyo.Constraint(rule=con_x)

## Solve

In [5]:
# opt = pyo.SolverFactory("ipopt", executable="/home/adb/anaconda3/bin/ipopt")
opt = pyo.SolverFactory("glpk", executable="/home/adb/anaconda3/bin/glpsol")
# opt = pyo.SolverFactory("glpk", executable="/usr/local/Caskroom/miniconda/base/bin/glpsol")

In [6]:
instance1 = model.create_instance()
results1 = opt.solve(instance1)

In [7]:
sol_dict1 = instance1.x.get_values()
sol1 = [k for k, v in sol_dict1.items() if v == 1]
not_sol1 = set([i for i in range(1, model.n+1)]) - set(sol1)
sol1_camm = ppd.county_ids_to_camm_ids(df, sol1)

print(f"Obtained Solution 1: \n{sol1}")
print(f"Obtained Solution 1 with Camm18 indexing method: \n{sol1_camm}")
print(f"Solution 1 from Camm18 paper: \n{known_x_sols[1]}")

Obtained Solution 1: 
[3, 5, 7, 8, 9, 28, 35, 40, 45, 49, 51, 72, 75, 76, 81]
Obtained Solution 1 with Camm18 indexing method: 
[4, 8, 14, 18, 29, 33, 35, 38, 51, 55, 59, 68, 75, 78, 87]
Solution 1 from Camm18 paper: 
[4, 8, 14, 18, 29, 33, 35, 38, 51, 55, 59, 68, 75, 78, 87]


## Re-solve for 2nd solution

In [8]:
def con_x_sol2(m):
    return sum(m.x[i] for i in sol1) - sum(m.x[j] for j in not_sol1) <= len(sol1) - 1
model.sol2_constraint = pyo.Constraint(rule=con_x_sol2)

instance2 = model.create_instance()
results2 = opt.solve(instance2)

In [9]:
sol_dict2 = instance2.x.get_values()
sol2 = [k for k, v in sol_dict2.items() if v == 1]
not_sol2 = set([i for i in range(1, model.n+1)]) - set(sol2)
sol2_camm = ppd.county_ids_to_camm_ids(df, sol2)

print(f"Obtained Solution 2: \n{sol2}")
print(f"Obtained Solution 2 with Camm18 indexing method: \n{sol2_camm}")
print(f"Solution 2 from Camm18 paper: \n{known_x_sols[2]}")

Obtained Solution 2: 
[3, 5, 7, 8, 9, 28, 35, 40, 45, 49, 51, 69, 72, 75, 76]
Obtained Solution 2 with Camm18 indexing method: 
[8, 14, 17, 18, 29, 33, 35, 38, 51, 55, 59, 68, 75, 78, 87]
Solution 2 from Camm18 paper: 
[8, 14, 17, 18, 29, 33, 35, 38, 51, 55, 59, 68, 75, 78, 87]


## Re-solve for 3rd solution

In [10]:
def con_x_sol3(m):
    return sum(m.x[i] for i in sol2) - sum(m.x[j] for j in not_sol2) <= len(sol2) - 1
model.sol3_constraint = pyo.Constraint(rule=con_x_sol3)

instance3 = model.create_instance()
results3 = opt.solve(instance3)

In [11]:
sol_dict3 = instance3.x.get_values()
sol3 = [k for k, v in sol_dict3.items() if v == 1]
not_sol3 = set([i for i in range(1, model.n+1)]) - set(sol3)
sol3_camm = ppd.county_ids_to_camm_ids(df, sol3)

print(f"Obtained Solution 3: \n{sol3}")
print(f"Obtained Solution 3 with Camm18 indexing method: \n{sol3_camm}")
print(f"Solution 3 from Camm18 paper: \n{known_x_sols[0]}")

Obtained Solution 3: 
[3, 5, 7, 8, 9, 26, 28, 40, 45, 49, 51, 69, 72, 75, 76]
Obtained Solution 3 with Camm18 indexing method: 
[8, 14, 17, 19, 29, 33, 35, 38, 51, 55, 59, 68, 75, 78, 87]
Solution 3 from Camm18 paper: 
[8, 14, 17, 19, 29, 33, 35, 38, 51, 55, 59, 68, 75, 78, 87]


## Verify

In [12]:
print(f"Solution 1 \nResult:{sol1_camm} \nCamm18:{known_x_sols[1]} \n ----")
print(f"Solution 2 \nResult:{sol2_camm} \nCamm18:{known_x_sols[2]} \n ----")
print(f"Solution 3 \nResult:{sol3_camm} \nCamm18:{known_x_sols[0]} \n ----")

Solution 1 
Result:[4, 8, 14, 18, 29, 33, 35, 38, 51, 55, 59, 68, 75, 78, 87] 
Camm18:[4, 8, 14, 18, 29, 33, 35, 38, 51, 55, 59, 68, 75, 78, 87] 
 ----
Solution 2 
Result:[8, 14, 17, 18, 29, 33, 35, 38, 51, 55, 59, 68, 75, 78, 87] 
Camm18:[8, 14, 17, 18, 29, 33, 35, 38, 51, 55, 59, 68, 75, 78, 87] 
 ----
Solution 3 
Result:[8, 14, 17, 19, 29, 33, 35, 38, 51, 55, 59, 68, 75, 78, 87] 
Camm18:[8, 14, 17, 19, 29, 33, 35, 38, 51, 55, 59, 68, 75, 78, 87] 
 ----


## Save solution for visualization

In [13]:
pd.Series(sol1).to_csv(output_path / "solution_model_1_sol1.csv", index=False, header=['county_id'])
pd.Series(sol2).to_csv(output_path / "solution_model_1_sol2.csv", index=False, header=['county_id'])
pd.Series(sol3).to_csv(output_path / "solution_model_1_sol3.csv", index=False, header=['county_id'])